In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

***Importing Libraries***

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
from sklearn import ensemble, tree, linear_model, preprocessing
import missingno as msno
import pandas_profiling
import plotly.express as px

Reading in the Data

In [ ]:
District_wise = pd.read_csv('../input/education-in-india/2015_16_Districtwise.csv')
State_wise_elementry = pd.read_csv('../input/education-in-india/2015_16_Statewise_Elementary.csv')
State_wise_secondary = pd.read_csv('../input/education-in-india/2015_16_Statewise_Secondary.csv')


In [ ]:
District_wise_met = pd.read_csv('../input/education-in-india/2015_16_Districtwise_Metadata.csv')
State_wise_elementry_met = pd.read_csv('../input/education-in-india/2015_16_Statewise_Elementary_Metadata.csv')
State_wise_secondary_met = pd.read_csv('../input/education-in-india/2015_16_Statewise_Secondary_Metadata.csv')


Going through the basic structure of Data.

In [ ]:
District_wise.head()

Going through the metadata of the data files.

In [ ]:
District_wise_met.head()

For doing a primitive analysis we are taking out the total number of schools in various categories.

In [ ]:
District_wise_total = pd.DataFrame()

In [ ]:
i=0
for name in District_wise_met['Description']:
    if 'Total' in name:
        District_wise_total[District_wise_met.iloc[i][1]] = District_wise[District_wise_met.iloc[i][0]]
    i=i+1

Checking how Schools are classified on the basis of their Ownership.

In [ ]:
District_wise_total['Schools_By_Category: Total']/(District_wise_total['Schools_by_Category:_Government: Total']+District_wise_total['Schools_by_Category:_Private_: Total']+District_wise_total['Schools_by_Category:_Madarsas_&_Unrecognised: Total'])

Therefore Goverment, Private and Unrecoganized are 3 catagories of Schools based on ownership.

Grouping the data frame on the basics of States.

In [ ]:
District_wise.head()

In [ ]:
District_wise_new = pd.DataFrame()

In [ ]:
District_wise_new['STATNAME'] = District_wise['STATNAME']

In [ ]:
District_wise_new['DISTNAME'] = District_wise['DISTNAME']

In [ ]:
District_wise_new = pd.concat([District_wise_new, District_wise_total], axis = 1 )

In [ ]:
District_wise_grouped = District_wise_new.groupby(by = 'STATNAME')

In [ ]:
State_wise_sum = District_wise_grouped.sum()

Finding out the sum various states data which is already grouped.

In [ ]:
State_wise_sum.head()

In [ ]:
State_wise_sum.index

Finding out what is the average number of persons avilable per school in various states.

In [ ]:
State_wise_sum['People_per_School'] = State_wise_sum['Basic_data_from_Census_2011: Total_Population(in_1000\'s)']/State_wise_sum['Schools_By_Category: Total']

In [ ]:
ax = sns.barplot(y=State_wise_sum.index, x='People_per_School', data = State_wise_sum  )
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
#plt.tight_layout()

plt.figure(figsize=(16,4))

This plot shows us the average number of people dependend per school in the particular state.

In [ ]:
State_wise_sum.head()

In [ ]:
State_wise_fac = pd.DataFrame()
for name in State_wise_sum.columns[21:31]:
    
    State_wise_fac[name] = State_wise_sum[name]

In [ ]:
State_wise_fac['Total_Schools'] = State_wise_sum['Schools_By_Category: Total']
State_wise_fac['Population_school_serves'] = State_wise_sum['People_per_School']

In [ ]:
for name in State_wise_fac.columns[0:8]:
    State_wise_fac[name] = State_wise_fac[name]/State_wise_fac['Total_Schools']

In [ ]:
State_wise_fac['Fraction_of_required_ramps'] = State_wise_fac['Schools_with_Ramp_(where_needed): Total']/State_wise_fac['Schools_where_Ramp_is_Required: Total']

In [ ]:
State_wise_fac = State_wise_fac.drop(['Schools_with_Ramp_(where_needed): Total','Schools_where_Ramp_is_Required: Total'], axis =1)

In [ ]:
State_wise_fac.head()

21 - All_whetherd roads 31 -- establishment

In [ ]:
pd.plotting.scatter_matrix(State_wise_fac, alpha = 0.3, figsize = (21,12), diagonal = 'kde')

In [ ]:
df_std

In [ ]:
df_std[:,1]

In [ ]:
State_wise_fac_scaled = pd.DataFrame()
i=0
for name in State_wise_fac.columns:
    State_wise_fac_scaled[name] = df_std[:,i]
    i=i+1

Schools_with_Computer: Total,Total_Schools,Population_school_serves

In [ ]:
State_wise_fac.head()

In [ ]:
temp = State_wise_fac
temp['Schools_with_Computer: Total'] = np.log(State_wise_fac['Schools_with_Computer: Total'])
temp['Total_Schools'] = np.log(State_wise_fac['Total_Schools'])
temp['Population_school_serves'] =  np.log(State_wise_fac['Population_school_serves'])

In [ ]:
pd.plotting.scatter_matrix(temp, alpha = 0.3, figsize = (21,12), diagonal = 'kde')

In [ ]:
temp.head()

In [ ]:
temp.drop(['Schools_with_Girls\'_Toilet: Total','Schools_with_Drinking_Water: Total'], inplace = True, axis=1)

In [ ]:
temp['Schools_with_Boys\'_Toilet: Total'] = np.arcsin(temp['Schools_with_Boys\'_Toilet: Total'])

In [ ]:
std_scale = preprocessing.StandardScaler().fit(temp)
df_std = std_scale.transform(temp)
df_std

In [ ]:
temp = pd.DataFrame()
i=0
for name in State_wise_fac.columns:
    temp[name] = df_std[:,i]
    i=i+1

In [ ]:
temp.head()

In [ ]:
pd.plotting.scatter_matrix(temp, alpha = 0.3, figsize = (21,12), diagonal = 'kde')

In [ ]:
plt.subplots(figsize=(12,9))
sns.heatmap(abs(temp.cov()), vmax=0.9)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=temp.shape[1]).fit(temp)


In [ ]:
#Fitting the PCA algorithm with our Data
pca = PCA().fit(temp)
#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
#plt.title('Pulsar Dataset Explained Variance')
plt.show()

In [ ]:
pca = PCA(n_components=5)
transformed_temp = pca.fit_transform(temp)

In [ ]:
transformed_temp[:,1]

In [ ]:
for_train = pd.DataFrame()
i=0
for name in ['1','2','3','4','5']:
    for_train[name] = transformed_temp[:,i]
    i=i+1

In [ ]:
for_train.head()

In [ ]:
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 

distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(for_train) 
    kmeanModel.fit(for_train)     
      
    distortions.append(sum(np.min(cdist(for_train, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / for_train.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(for_train, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / for_train.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 

In [ ]:
for key,val in mapping1.items(): 
    print(str(key)+' : '+str(val)) 

In [ ]:

plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show() 